# Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sp
from joblib import dump, load
from collections import namedtuple

%matplotlib inline
plt.style.use('seaborn')
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

from google.colab import drive
import os
%load_ext autoreload
%autoreload 2

drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis')

Mounted at /content/gdrive


In [ ]:
!pip install antspyx

     |████████████████████████████████| 301.6MB 57kB/s 
     |████████████████████████████████| 71kB 7.4MB/s 


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install deepbrain

     |████████████████████████████████| 686kB 12.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e92b4788320dd490e5156fb55b33b2418c709d390cccebb975913694748d5e85
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.2 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0


# Preprocessing and Reprocessing MRIs Faster with multiple notebooks

In [3]:
os.chdir('/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/src/data_preprocessing/')
from mri_preprocessing import *

output_path = "/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210523/"
input_path='/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI'

In [ ]:
images_to_process,_,_ = list_available_images(input_path)
images_to_process[:10]

Looking for MRI images in path: /content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI 

Found a total of  1202  images.
Found a total of  0  mask images.
Available images to process:  1202 



['/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/057_S_0464/MPR-R__GradWarp__B1_Correction__N3__Scaled/2009-06-24_13_49_46.0/S69136/ADNI_057_S_0464_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20091203185358387_S69136_I160245.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/036_S_0673/MPR__GradWarp__B1_Correction__N3__Scaled/2006-07-21_08_45_17.0/S17157/ADNI_036_S_0673_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070120001203137_S17157_I36949.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/033_S_0514/MPR__GradWarp__B1_Correction__N3__Scaled/2007-06-07_15_43_00.0/S33320/ADNI_033_S_0514_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070809185451369_S33320_I66544.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/013_S_0502/MPR__GradWarp__B1_Correction__N3__Scal

In [ ]:
raw_image_names = [x.split(sep='/')[-1] for x in images_to_process]
processed_images,_,_ = list_available_images(output_path,file_format='.nii.gz')
processed_image_names = [x.split(sep='/20210523/')[-1].split('.gz')[0] for x in processed_images]
image_names_to_reprocess = list(set(raw_image_names) - set(processed_image_names))
images_to_reprocess = [x for x in images_to_process if x.split(sep='/')[-1] in image_names_to_reprocess]
len(images_to_reprocess)

Looking for MRI images in path: /content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210523/ 

Found a total of  1658  images.
Found a total of  0  mask images.
Available images to process:  1658 



344

## Split image array

In [ ]:
images_to_reprocess = images_to_process[900:1050]

## Execute

In [ ]:
execute_preprocessing(output_path=output_path, 
                      images_to_process = images_to_reprocess,
                      box=100)

Setting ANTs and NiftyReg environment variables...

------------------------------------------------------------------------------------------------------------------------
Starting pre-processing (Labeling + Standardizing + Registration + Skull Stripping + Cropping) for 150 images. This might take a while... =)
Processing from  image 0 to image 0.
------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------
Processing image (1/150): /content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/116_S_0370/MPR__GradWarp__N3__Scaled/2008-05-08_13_09_27.0/S49840/ADNI_116_S_0370_MR_MPR__GradWarp__N3__Scaled_Br_20080522134715771_S49840_I106529.nii
Standardizing image based on Atlas...
Found a total of 0 NaN values in image.
Registering image to Atlas...
Stripping skull from image..

# Final MRI Preparations

In [ ]:
os.chdir('/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/src/data_preparation/')
from mri_preparation import *

input_path='/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210523/'
input_path='/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210523/'

In [ ]:
execute_mri_data_preparation(input_path,output_path,orientation = 'axial',orientation_slice = 25,num_augmented_images = 5,sampling_range = 5,file_format = '.nii.gz')